#Part 1

##Loading data

In [1]:
import urllib.request
urllib.request.urlretrieve('https://drive.google.com/uc?export=download&confirm=t&id=1FrEC8j4VcrQ8Od12M5HZOSNHDGLoivGV', "user-shows.txt")
urllib.request.urlretrieve('https://drive.google.com/uc?export=download&confirm=t&id=1Oghwz6LcxRblphATjA3Ela-1tuOgYWza', "shows.txt")
urllib.request.urlretrieve('https://drive.google.com/uc?export=download&confirm=t&id=1Jj_uDE8zHZqZiHOZFPqD5XnwOrz2Jwye', "graph-small.txt")
urllib.request.urlretrieve('https://drive.google.com/uc?export=download&confirm=t&id=1JJ78XF-93WWQC8OfdivPpoWzov4eU5FK', "graph-full.txt")


('graph-full.txt', <http.client.HTTPMessage at 0x7eff5a9911e0>)

In [2]:
test = "1 1 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 0 0 1 1 1 0 0 1 0 0 1 1 1 1 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 0 0 0 1 1 0 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 0 0 1 1 1 0 0 1 1 1 1 1 0 1 1 0 1 1 0 0 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 0 0 0 0 1 1 0 1 1 1 0 1 1 1 0 0 0 1 0 1 1 1 1 0 1 1 1 1 0 1 0 1 1 0 1 0 0 1 1 1 0 1 0 0 1 1 1 1 0 1 1 1 1 1 0 0 0 0 1 0 1 1 1 1 1 1 1 1 0 0 0 1 1 0 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 1 0 0 0 1 1 1 1 1 1 0 0 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1 0 1 0 1 1 0 1 0 1 1 0 0 0 0 1 1 1 1 0 0 1 1 0 0 1 1 1 1 1 0 1 1 0 1 0 1 1 0 1 0 1 1 1 1 1 0 1 1 1 0 1 0 0 1 1 0 1 0 0 1 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 0 0 1 0 0 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1 1 0 0 1 1 1 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 1 1 1 0 1 1 1 1 1 1 0 0 0 0 0 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 0 1 0 1 0 1 1 1 1 0 1 1 1 0 1 0 1 1 0 0 1 0 1 1 1 0 1 0 0 1 1 1 0 1 1 1 0 0 1 0 0 1 0 1 1 1 1 0 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 0 1 0 0"
test_user_shows = [int(x) for x in test.split()]


In [3]:
%env PYTHONHASHSEED 3
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install -q pyspark

env: PYTHONHASHSEED=3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [4]:
from math import sqrt
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import *
import numpy as np

spark = SparkSession.builder.master("local[*]").appName('SparkExample').config(
    "spark.executor.memory", "1g").config("spark.ui.port", "4050"
        ).getOrCreate()
sc = spark.sparkContext

def dbg(x):
  """ A helper function to print debugging information on RDDs """
  if isinstance(x, pyspark.RDD):
    print([(t[0], list(t[1]) if 
            isinstance(t[1], pyspark.resultiterable.ResultIterable) else t[1])
           if isinstance(t, tuple) else t
           for t in x.take(100)])
  else:
      print(x) 

def to_list(x):
  """ A helper function to convert nested pyspark.resultiterable.ResultIterable iterables to lists """
  if hasattr(x, '__iter__'):
    if isinstance(x, pyspark.resultiterable.ResultIterable):
      return list(x)
    else:
      return [to_list(t) for t in x]
  else:
      return x

def isclose(x, y):
  """ A helper function to compare if two nested items are close numerically if they are floating point"""
  try:
    if hasattr(x, '__iter__'):
        return all(isclose(x[i], y[i]) for i in (x.keys() if isinstance(x, dict) else range(len(x))))
    else:
      if np.isreal(x):
        return np.isclose(x, y)
      else:
        return x == y
  except:
    return False

In [5]:

user_shows_file = sc.textFile("user-shows.txt")
user_shows_index = user_shows_file.zipWithIndex().map(lambda kv: (kv[1],kv[0]))
dbg(user_shows_index)
print(user_shows_index.count())

shows_file = sc.textFile("shows.txt")
shows_index = shows_file.zipWithIndex().map(lambda kv: (kv[1],kv[0]))
dbg(shows_index)
print(shows_index.count())

user_shows = user_shows_index.map(lambda kv: (kv[0], [int(x) for x in kv[1].split()]))
dbg(user_shows)

[(0, '1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

##1.1.a

In [6]:
def cosine_similarity(u, v):
  numerator, squared_u, squared_v = 0,0,0
  for i in range(len(u)):
    numerator += u[i]*v[i]
    squared_u += u[i]**2
    squared_v += v[i]**2
  denominator = sqrt(squared_u) * sqrt(squared_v)
  return numerator/denominator

print(cosine_similarity([1,2,-1],[2,1,1]))
print(cosine_similarity([1,2,-1],[1,2,1]))
print(cosine_similarity([1,2,1],[1,2,1]))
print(cosine_similarity([1, 1, 0, 1, 1, 0], [0, 0, 0, 0, 1, 1]))

0.5000000000000001
0.6666666666666667
1.0000000000000002
0.35355339059327373


In [7]:
assert isclose(cosine_similarity([1,2,-1],[2,1,1]), 0.5)
assert isclose(cosine_similarity([1,2,-1],[1,2,1]), 0.6666666666666667)
assert isclose(cosine_similarity([1,2,1],[1,2,1]), 1.0)
assert isclose(cosine_similarity([1, 1, 0, 1, 1, 0], [0, 0, 0, 0, 1, 1]), 0.35355339059327373)


##1.1.b

In [8]:
def q1b(test_user_shows):
  similarity = user_shows.map(lambda x : (x[0], cosine_similarity(x[1], test_user_shows)))
  return similarity
print(q1b(test_user_shows).takeOrdered(5, lambda x: -x[1]))

[(6733, 0.5653963756883335), (1572, 0.5611939760061074), (2907, 0.5580880342929785), (5623, 0.5554706066152114), (2408, 0.5496782626513883)]


In [9]:
assert isclose([x[1] for x in q1b(test_user_shows).takeOrdered(5, lambda x: -x[1])], 
               [0.5653963756883335,0.5611939760061074,0.5580880342929785,0.5554706066152114,0.5496782626513883])

In [10]:
def q1c(test_user_shows):
    similar_5_users = q1b(test_user_shows).sortBy(lambda x : -x[1]).map(lambda x : x[0]).take(5)
    similar_users_shows = user_shows.filter(lambda x: x[0] in similar_5_users)
    similar_users_shows_index = similar_users_shows.flatMap(lambda x : [(i, x[1][i]) for i in range(len(x[1]))])
    all_similar_users_watched = similar_users_shows_index.reduceByKey(lambda a,b : a+b).filter(lambda x : x[1] == 5)
   
    recommended_shows = all_similar_users_watched.filter(lambda x : test_user_shows[x[0]] != 1)
    shows_name = recommended_shows.join(shows_index).map(lambda x : x[1][1])
    return shows_name

print(q1c(test_user_shows).collect())
# ['"Two and a Half Men"', '"The Oprah Winfrey Show"', '"The Early Show"', '"Family Guy"', '"George Lopez"', '"Today"', '"Everybody Loves Raymond"']

['"Today"', '"Everybody Loves Raymond"', '"Two and a Half Men"', '"The Oprah Winfrey Show"', '"The Early Show"', '"Family Guy"', '"George Lopez"']


In [11]:
assert set(q1c(test_user_shows).collect()) == set(['"Two and a Half Men"', '"The Oprah Winfrey Show"', '"The Early Show"', '"Family Guy"', '"George Lopez"', '"Today"', '"Everybody Loves Raymond"'])

##1.2.a

In [12]:
def q2a(filename):
    read_file = sc.textFile(filename)
    src_dst_split = read_file.map(lambda x: x.split())
    zero_based = src_dst_split.map(lambda x: (int(x[0])-1, int(x[1])-1))
    grouped_by_src = zero_based.groupByKey()
    return grouped_by_src

In [13]:
assert to_list(q2a("graph-small.txt").sortByKey().take(3)) == \
  [[0, [1, 7, 7, 13, 13, 21, 25, 43, 52, 65, 66, 77, 77, 78, 79, 81, 94]], 
   [1, [2, 6, 12, 52, 52, 57, 60, 65, 89]], 
   [2, [0, 3, 13, 19, 20, 34, 37, 47, 52, 52, 65, 71, 77, 82]]]


##1.2.b

In [14]:
def lis_to_dict(given_list):
    unique_dst = len(set(given_list))
    dst_dict = {}
    for dst in given_list:
      dst_dict[dst] = 1/unique_dst
    return dst_dict
    
def q2b(graph_rdd):
    dst_to_dict = graph_rdd.map(lambda x : (x[0], lis_to_dict(x[1])))
    return dst_to_dict

dbg(q2b(q2a("graph-small.txt")).sortByKey().take(3))

[(0, {1: 0.07142857142857142, 7: 0.07142857142857142, 13: 0.07142857142857142, 21: 0.07142857142857142, 25: 0.07142857142857142, 43: 0.07142857142857142, 52: 0.07142857142857142, 65: 0.07142857142857142, 66: 0.07142857142857142, 77: 0.07142857142857142, 78: 0.07142857142857142, 79: 0.07142857142857142, 81: 0.07142857142857142, 94: 0.07142857142857142}), (1, {2: 0.125, 6: 0.125, 12: 0.125, 52: 0.125, 57: 0.125, 60: 0.125, 65: 0.125, 89: 0.125}), (2, {0: 0.07692307692307693, 3: 0.07692307692307693, 13: 0.07692307692307693, 19: 0.07692307692307693, 20: 0.07692307692307693, 34: 0.07692307692307693, 37: 0.07692307692307693, 47: 0.07692307692307693, 52: 0.07692307692307693, 65: 0.07692307692307693, 71: 0.07692307692307693, 77: 0.07692307692307693, 82: 0.07692307692307693})]


In [15]:
assert isclose(q2b(q2a("graph-small.txt")).sortByKey().take(3),
               [(0, {1: 0.07142857142857142, 65: 0.07142857142857142, 66: 0.07142857142857142, 
                    7: 0.07142857142857142, 43: 0.07142857142857142, 13: 0.07142857142857142, 
                    77: 0.07142857142857142, 78: 0.07142857142857142, 79: 0.07142857142857142, 
                    81: 0.07142857142857142, 52: 0.07142857142857142, 21: 0.07142857142857142, 
                    25: 0.07142857142857142, 94: 0.07142857142857142}), 
                (1, {89: 0.125, 65: 0.125, 2: 0.125, 6: 0.125, 12: 0.125, 52: 0.125, 57: 0.125, 60: 0.125}),
                (2, {0: 0.07692307692307693, 65: 0.07692307692307693, 34: 0.07692307692307693, 
                    3: 0.07692307692307693, 37: 0.07692307692307693, 71: 0.07692307692307693, 
                    13: 0.07692307692307693, 77: 0.07692307692307693, 47: 0.07692307692307693, 
                    82: 0.07692307692307693, 19: 0.07692307692307693, 20: 0.07692307692307693, 
                    52: 0.07692307692307693})])

##1.2.c

In [16]:
def q2c(transition_matrix_col_rdd):
    flatten = transition_matrix_col_rdd.flatMap(lambda x : [(i, (x[0],x[1][i])) for i in x[1]])
    sorted_by_row = flatten.groupByKey()
    return sorted_by_row
    
dbg(to_list(q2c(q2b(q2a("graph-small.txt"))).map(lambda kv: (kv[0],sorted(kv[1]))).sortByKey().take(3)))

[[0, [[2, 0.07692307692307693], [10, 0.07142857142857142], [12, 0.1111111111111111], [13, 0.09090909090909091], [16, 0.08333333333333333], [23, 0.1111111111111111], [24, 0.08333333333333333], [27, 0.1111111111111111], [29, 0.1111111111111111], [30, 0.06666666666666667], [32, 0.08333333333333333], [37, 0.09090909090909091], [43, 0.1111111111111111], [45, 0.07692307692307693], [49, 0.09090909090909091], [50, 0.14285714285714285], [52, 0.25], [55, 0.1111111111111111], [57, 0.06666666666666667], [58, 0.058823529411764705], [63, 0.125], [64, 0.1111111111111111], [72, 0.125], [78, 0.07142857142857142], [81, 0.08333333333333333], [84, 0.1], [87, 0.125], [88, 0.08333333333333333], [93, 0.1111111111111111], [97, 0.16666666666666666], [99, 0.09090909090909091]]], [1, [[0, 0.07142857142857142], [25, 0.14285714285714285], [44, 0.0625], [57, 0.06666666666666667]]], [2, [[1, 0.125], [11, 0.1], [25, 0.14285714285714285], [66, 0.09090909090909091], [82, 0.06666666666666667], [85, 0.14285714285714285],

In [17]:
test_q2c = to_list(q2c(q2b(q2a("graph-small.txt"))).map(lambda kv: (kv[0],sorted(kv[1]))).sortByKey().take(3))
assert isclose(test_q2c,
  [[0, [[2, 0.07692307692307693], [10, 0.07142857142857142], [12, 0.1111111111111111], 
        [13, 0.09090909090909091], [16, 0.08333333333333333], [23, 0.1111111111111111], 
        [24, 0.08333333333333333], [27, 0.1111111111111111], [29, 0.1111111111111111], 
        [30, 0.06666666666666667], [32, 0.08333333333333333], [37, 0.09090909090909091], 
        [43, 0.1111111111111111], [45, 0.07692307692307693], [49, 0.09090909090909091], 
        [50, 0.14285714285714285], [52, 0.25], [55, 0.1111111111111111], [57, 0.06666666666666667], 
        [58, 0.058823529411764705], [63, 0.125], [64, 0.1111111111111111], [72, 0.125], 
        [78, 0.07142857142857142], [81, 0.08333333333333333], [84, 0.1], [87, 0.125], 
        [88, 0.08333333333333333], [93, 0.1111111111111111], [97, 0.16666666666666666], 
        [99, 0.09090909090909091]]], 
   [1, [[0, 0.07142857142857142], [25, 0.14285714285714285], [44, 0.0625], 
        [57, 0.06666666666666667]]],
   [2, [[1, 0.125], [11, 0.1], [25, 0.14285714285714285], [66, 0.09090909090909091], 
        [82, 0.06666666666666667], [85, 0.14285714285714285], [95, 0.09090909090909091]]]])


##1.2.d

In [18]:
def row_multiply(row, R):
  matrix_multiplication = 0
  for i, mul_val in row:
    matrix_multiplication += R[i]*mul_val
  return matrix_multiplication

In [19]:
assert isclose(row_multiply([(0,1),(2,1)],[.2,.3,.5,0]), 0.7)
assert isclose(row_multiply([(1,1),(3,5)],[.2,.3,.5,0]), 0.3)
assert isclose(row_multiply([(0,.1),(1,.1),(2,.1),(3,.1)],[.2,.3,.5,0]), 0.1)

## 1.2.e
You don't need to write any code here, just run the below after passing all of the tests above.

In [20]:
def q2e(filename):
  graph_rows = q2c(q2b(q2a(filename)))
  N = graph_rows.count()
  R = dict(enumerate([1/N]*N))
  # R is technically not a vector; it is a dictionary of index to value
  for t in range(100):
    #print(sorted(R.items()))
    vecR = sc.broadcast(R)
    #compute R again
    row_results = graph_rows.map(lambda kv: (kv[0], row_multiply(kv[1],vecR.value)))
    R = row_results.collectAsMap()
    
  #print("R:",sorted(R.items()))
  return row_results.sortBy(lambda kv: -kv[1]).take(5)

print(q2e("graph-small.txt"))

[(52, 0.04243140406575616), (39, 0.039884267610139156), (13, 0.039377727003573254), (0, 0.03540059180244296), (26, 0.03489656776403212)]


In [21]:
assert isclose(q2e("graph-small.txt"),
               [(52, 0.04243140406575616),
                (39, 0.039884267610139156),
                (13, 0.039377727003573254),
                (0, 0.03540059180244296),
                (26, 0.03489656776403212)])

In [22]:
print(q2e("graph-full.txt"))


[(536, 0.0023177681473929425), (262, 0.0022954975147433765), (964, 0.002189010552384874), (242, 0.002097424249330295), (254, 0.0020787314135073216)]


In [23]:
import urllib.request
urllib.request.urlretrieve('https://drive.google.com/uc?export=download&confirm=t&id=1Jj_uDE8zHZqZiHOZFPqD5XnwOrz2Jwye', "graph-small.txt")
urllib.request.urlretrieve('https://drive.google.com/uc?export=download&confirm=t&id=1JJ78XF-93WWQC8OfdivPpoWzov4eU5FK', "graph-full.txt")
urllib.request.urlretrieve('https://drive.google.com/uc?export=download&confirm=t&id=1bIaOO3SiLvim3gg5Dv7r7QcyxBLNuKkd', "facebook-medium.txt")
urllib.request.urlretrieve('https://drive.google.com/uc?export=download&confirm=t&id=1__W1Sc4-QWHYRcsGsryMG9ZbzNMN98IL', "facebook-large.txt")

('facebook-large.txt', <http.client.HTTPMessage at 0x7eff422d20e0>)

In [24]:
#dbg(to_list(q2b(q2a("facebook-large.txt")).sortByKey().take(3)))
print(q2e("facebook-large.txt"))


[(426, 0.00279964035585908), (427, 0.002776375218395057), (433, 0.002708151848806963), (438, 0.0026984926092680096), (434, 0.002685260079087062)]
